# Deploy to ACI

In [1]:
import os
import json

from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

## One-time auth

In [2]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

ws = Workspace.get(name='nlp-ml', 
                subscription_id='50324bce-875f-4a7b-9d3c-0e33679f5d72', 
                resource_group='nlp',
               auth=interactive_auth)

## Parameters

In [3]:
language = 'de'
env = 'test'
version = '0.2'

#NOTE:
# Change language setting in custom.py before deployment!

In [4]:
# ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

nlp-ml
nlp
westeurope
50324bce-875f-4a7b-9d3c-0e33679f5d72


## Upload Model Assets

In [5]:
# model = Model.register(workspace=ws,
#                        model_name=f'nlp_{language}_{env}',                # Name of the registered model in your workspace.
#                        model_path=f'../assets/assets_{language}.zip',  # Local file to upload and register as a model.
#                        resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=4),
#                        description='NLP test version',
#                        tags={'models': 'classification, ner, qa', 'version': version,
#                              'language': language, 'environment': env})

# print('Name:', model.name)
# print('Version:', model.version)

In [6]:
model = Model(ws, f'nlp_{language}_{env}')


In [7]:
print('Name:', model.name)
print('Version:', model.version)

Name: nlp_de_test
Version: 1


## Configure Compute

In [8]:
os.chdir('..')

In [9]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4) #, auth_enabled=True)

In [10]:
environment = Environment('farmenv')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
                                                                                'azureml-defaults',
                                                                                'spacy',
                                                                                'transformers==2.3.0',
                                                                                'scipy',
                                                                                'numpy',
                                                                                'azure-storage-blob',
                                                                                'tqdm',
                                                                                'boto3',
                                                                                'scipy>=1.3.2',
                                                                                'sklearn',
                                                                                'seqeval',
                                                                                'mlflow==1.0.0',
                                                                                'dotmap==1.3.0',
                                                                                'git+https://github.com/deepset-ai/FARM.git',
                                                                                'git+https://github.com/zalandoresearch/flair.git'
                                                                            ],
                                                                 conda_packages=[
                                                                                'pytorch',
                                                                                'torchvision',
                                                                                'gensim',
                                                                                'numpy',
                                                                                'pandas'
                                                                            ])

In [11]:
environment.python.conda_dependencies.serialize_to_string()

'# Conda environment specification. The dependencies defined in this file will\r\n# be automatically provisioned for runs with userManagedDependencies=False.\r\n\n# Details about the Conda environment file format:\r\n# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually\r\n\nname: project_environment\ndependencies:\n  # The python interpreter version.\r\n  # Currently Azure ML only supports 3.5.2 and later.\r\n- python=3.6.2\n\n- pip:\n  - azureml-defaults==1.0.79.*\n  - spacy\n  - transformers==2.3.0\n  - numpy\n  - azure-storage-blob\n  - tqdm\n  - boto3\n  - scipy>=1.3.2\n  - sklearn\n  - seqeval\n  - mlflow==1.0.0\n  - dotmap==1.3.0\n  - git+https://github.com/deepset-ai/FARM.git\n  - git+https://github.com/zalandoresearch/flair.git\n- pytorch\n- torchvision\n- gensim\n- numpy\n- pandas\nchannels:\n- conda-forge\n'

## Deploy

In [13]:
service_name = f'nlp-{language}-{env}'

inference_config = InferenceConfig(entry_script='code/infer.py',
                                   source_directory='.',
                                   environment=environment)

# Remove any existing service under the same name.
try:
#     Webservice(ws, service_name).delete()
    # Retrieve existing service.
    service = Webservice(name=service_name, workspace=ws)
    print('[INFO] Updating web service')
    # Update to new model(s).
    service.update(models=[], inference_config=inference_config)
except WebserviceException:
    print('[INFO] Creating web service')
    service = Model.deploy(ws, service_name, [], inference_config, deployment_config=aci_config) #, overwrite=True)
service.wait_for_deployment(show_output=True)

[INFO] Creating web service


KeyboardInterrupt: 

In [ ]:
service.get_keys

In [ ]:
service.get_logs()

## Test Service

In [ ]:
service.run(json.dumps([{"body": "Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?"}]))

In [ ]:
service.run(json.dumps([{"subject": "My pc won't start:","body": "My pc won't start:"}]))